In [12]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import dill
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder
import os

In [13]:
with open('read_file.pickle', 'rb') as f: 
    load_data = dill.load(f)


df_test = load_data('test.csv')

In [14]:
with open('drop_features.pickle', 'rb') as f:
    drop_features = dill.load(f)

df_test = drop_features(df_test, features_to_drop=['Id','Alley','PoolQC','Fence','MiscFeature'])


In [15]:
df_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,32,0,0,0,0,700,7,2006,WD,Normal


In [16]:
with open('clean_data.pickle', 'rb') as f:
    clean_data = dill.load(f)

df_test = clean_data(df_test, train=False)

In [17]:
df_test.isna().sum().sum()

np.int64(0)

In [18]:
with open('encode_data.pickle', 'rb') as f:
    encode_data = dill.load(f)

Target_Encoding_list= ['MSZoning', 'Street', 'Utilities', 'LotConfig', 'Neighborhood', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'SaleType']
Ordinal_Encoding_list = ['LotShape', 'MasVnrType', 'LandContour', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish',  'GarageQual',  'GarageCond', 'PavedDrive', 'SaleCondition', 'LotConfig']

encoding_methods = {col: 'target' for col in Target_Encoding_list} 
encoding_methods.update({col: 'ordinal' for col in Ordinal_Encoding_list})

df = encode_data(df_test, encoding_methods , train=False, target='SalePrice')


In [19]:
with open('train_columns.pickle', 'rb') as f: 
    train_columns = dill.load(f)
    
df_test.columns == train_columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [20]:
with open('predict_model.pickle', 'rb') as f:
    pred = dill.load(f)

with open('trained_model.pickle', 'rb') as f: 
    model = dill.load(f)


with open('train_columns.pickle', 'rb') as f: 
    train_columns = dill.load(f)

df_test = df_test[train_columns] 
for col in train_columns: 
    df_test[col] = df_test[col].astype(float)


y_new_pred = pred(df_test, model)

Model's raw prediction: [-0.87356328 -0.17431282  0.01316399 ... -0.24669526 -0.85112379
  1.03553954]
Predictions after inverse transform (if applicable):[117497.59002431 154600.90919495 166570.74848232 ... 150230.28545548
 118526.32486278 252092.60640884]


In [21]:
y_new_pred

array([117497.59002431, 154600.90919495, 166570.74848232, ...,
       150230.28545548, 118526.32486278, 252092.60640884])